#### Database operations in Pandas

In [1]:
import pandas as pd

__Query__

In [7]:
# load the data
df = pd.read_csv('../datasets/nyc_weather_2018.csv')

In [3]:
df.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


In [4]:
'''
SELECT *
FROM weather
WHERE datatype = 'SNOW' AND value > 0;
'''
# in pandas query use '==' unlike '=' in where clause of sql
snow_data = df.query('datatype == "SNOW" and value > 0')

In [5]:
snow_data.head()

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
699,2018-01-04T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",229.0
702,2018-01-04T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",10.0
706,2018-01-04T00:00:00,SNOW,GHCND:US1NJBG0018,",,N,",46.0
713,2018-01-04T00:00:00,SNOW,GHCND:US1NJES0018,",,N,",10.0


In [6]:
# compare the standard filtering vs query
# query is preferable when the name of the data frame is pretty long
df[(df.datatype == 'SNOW') & (df.value > 0)].equals(snow_data)

True

__Merging dataframes__

In [8]:
stations_info = pd.read_csv('../datasets/weather_stations.csv')
stations_info.head()

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6


In [8]:
stations_info.id.describe()

count                   279
unique                  279
top       GHCND:US1CTFR0022
freq                      1
Name: id, dtype: object

In [9]:
df.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

In [10]:
# check rows of the data frames
df.shape[0], stations_info.shape[0]

(78780, 279)

In [11]:
# star doesn't limit to 1 data frame and allows to pass as many as we need
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(df, stations_info)

[78780, 279]

In [12]:
# more efficient function
def get_info(attr: str, *dfs):
    ''' 
    attr: data frame attribute
    dfs: data frames
    returns: list with the attribute information
    '''
    return list(map(lambda x: getattr(x, attr), dfs))

get_info('shape', df, stations_info)

[(78780, 5), (279, 5)]

In [13]:
# default join
inner_join = df.merge(stations_info, left_on='station', right_on='id')
inner_join.sample(5)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
27644,2018-06-13T00:00:00,WESF,GHCND:US1NYWC0018,",,N,",0.0,GHCND:US1NYWC0018,"ARMONK 0.3 SE, NY US",41.12996,-73.708161,117.3
60972,2018-05-22T00:00:00,WSF2,GHCND:USW00094741,",,W,",5.8,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",40.85000,-74.061390,2.7
59268,2018-10-17T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",7.8,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.969250,42.7
42991,2018-06-15T00:00:00,SNOW,GHCND:USW00014732,",,W,",0.0,GHCND:USW00014732,"LAGUARDIA AIRPORT, NY US",40.77944,-73.880350,3.4
58900,2018-09-09T00:00:00,WDF5,GHCND:USW00094728,",,W,",60.0,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.969250,42.7


In [14]:
# remove duplicates id and station
df.merge(stations_info.rename(dict(id='station'), axis=1), on='station').sample(5)

,date,datatype,station,attributes,value,name,latitude,longitude,elevation
23412,2018-11-23T00:00:00,SNOW,GHCND:US1NYNS0042,",,N,",0.0,"ALBERTSON 0.2 SSE, NY US",40.769131,-73.647484,43.3
52217,2018-10-23T00:00:00,TMAX,GHCND:USW00054743,",,W,",17.8,"CALDWELL ESSEX CO AIRPORT, NJ US",40.876390,-74.283060,52.7
57654,2018-05-18T00:00:00,WSF2,GHCND:USW00094728,",,W,",7.2,"NY CITY CENTRAL PARK, NY US",40.778980,-73.969250,42.7
36214,2018-04-18T00:00:00,TOBS,GHCND:USC00289187,",,7,0700",0.0,"WANAQUE RAYMOND DAM, NJ US",41.044400,-74.293300,74.7
56788,2018-02-26T00:00:00,WT01,GHCND:USW00094728,",,W,",1.0,"NY CITY CENTRAL PARK, NY US",40.778980,-73.969250,42.7


In [16]:
left_join = stations_info.merge(df, left_on='id', right_on='station', how='left')
# get the same data with right join and switching the df's and left/right indexes
right_join = df.merge(stations_info, left_on='station', right_on='id', how='right')

In [18]:
left_join.head(2)

,id,name,latitude,longitude,elevation,date,datatype,station,attributes,value
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6,NaN,NaN,NaN,NaN,NaN
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0


In [19]:
right_join.tail(2)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
78947,2018-12-31T00:00:00,WT01,GHCND:USW00094789,",,W,",1.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63915,-73.76401,3.4
78948,2018-12-31T00:00:00,WT02,GHCND:USW00094789,",,W,",1.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63915,-73.76401,3.4


In [25]:
left_join.sort_index(axis=1).sort_values(
    ['date', 'station']).reset_index().drop(
    columns='index'
).equals(
    right_join.sort_index(axis=1).sort_values(
        ['date', 'station']).reset_index().drop(
    columns='index'
)
)

True

In [26]:
get_info('shape', inner_join, left_join, right_join)

[(78780, 10), (78949, 10), (78949, 10)]

_Full outer join_

In [27]:
stations_info.columns

Index(['id', 'name', 'latitude', 'longitude', 'elevation'], dtype='object')

In [30]:
df.columns

Index(['date', 'datatype', 'station', 'attributes', 'value'], dtype='object')

In [39]:
# indicator = True adds the column _merge that shows the type of merge applied
outer_join = df.merge(
    stations_info[stations_info.name.str.contains('NY')],
    left_on='station', right_on='id', how='outer', indicator=True
    )

In [40]:
# result of indicator=True
outer_join.sample(10)._merge

26529         both
3992     left_only
33816    left_only
60611    left_only
22203         both
8313     left_only
78438    left_only
65611         both
69033         both
45803    left_only
Name: _merge, dtype: category
Categories (3, object): ['left_only', 'right_only', 'both']

In [44]:
outer_join.sample(2).append(
    outer_join[outer_join.station.isna()].head(2)
    ).append(
        outer_join[outer_join.id.isna()].head(2)
    )

/var/folders/1h/vmdz8n3d00lfsp1hw8300jt00000gn/T/ipykernel_37139/2583366133.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outer_join.sample(2).append(
/var/folders/1h/vmdz8n3d00lfsp1hw8300jt00000gn/T/ipykernel_37139/2583366133.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ).append(


,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation,_merge
36089,2018-03-18T00:00:00,PRCP,GHCND:USC00289187,",,7,0700",0.0,NaN,NaN,NaN,NaN,NaN,left_only
49891,2018-02-05T00:00:00,TMIN,GHCND:USW00054743,",,W,",-7.1,NaN,NaN,NaN,NaN,NaN,left_only
78780,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJMS0036,"PARSIPPANY TROY HILLS TWP 2.1 E, NJ US",40.865600,-74.385100,64.3,right_only
78781,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJMS0039,"PARSIPPANY TROY HILLS TWP 1.3 WSW, NJ US",40.853314,-74.446957,94.2,right_only
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0,NaN,NaN,NaN,NaN,NaN,left_only
1,2018-01-02T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0,NaN,NaN,NaN,NaN,NaN,left_only


In [2]:
dirty_data = pd.read_csv('../datasets/dirty_data.csv', index_col='date'
    ).drop_duplicates().drop(columns='SNWD')
dirty_data.head(2)

,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False


In [3]:
valid_station = dirty_data.query('station != "?"').copy().drop(
    columns=['WESF', 'station']
)
station_with_wesf = dirty_data.query('station == "?"').copy().drop(
    columns=['station', 'TOBS', 'TMIN', 'TMAX']
)

In [4]:
# now we are going to merge on index, so we pass left_index and right_index equal Truee
valid_station.merge(station_with_wesf,left_index=True, right_index=True).query(
    'WESF > 0'
).head()
# _x : left dataframe
# _y : right dataframe

,PRCP_x,SNOW_x,TMAX,TMIN,TOBS,inclement_weather_x,PRCP_y,SNOW_y,WESF,inclement_weather_y
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


To specify your own suffix use `suffixes` argument

In [5]:
valid_station.merge(
    station_with_wesf,left_index=True, right_index=True,
    suffixes=['','_?']
    ).query(
    'WESF > 0'
).head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


When joining on index it makes more sence to use `join` instead of `merge`. The suffixes are now `rsuffix` and `lsuffix`

In [6]:
valid_station.join(station_with_wesf, rsuffix='_?').query(
    'WESF > 0'
).head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


Using join can be very resource-intensive. To know upfront what will happen to the dataframes with inner join, we can use the __index *differencing*__ and __*intersection*__.

In [9]:
df.head(1)

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0


In [10]:
stations_info.head(1)

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.0641,-73.577,36.6


In [11]:
df.set_index('station', inplace=True)
stations_info.set_index('id', inplace=True)

In [13]:
# how many same indeces? -> length 110
df.index.intersection(stations_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

In [14]:
df.index.difference(stations_info.index)

Index([], dtype='object')

In [15]:
stations_info.index.difference(df.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

We don't use anything from the `df` dataframe, but we do loose 169 elements from the `stations_info`

`symmetric_difference`

In [16]:
df.index.symmetric_difference(stations_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

In [18]:
stations_info.index.symmetric_difference(df.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

In [22]:
(df.index.symmetric_difference(stations_info.index) ==\
    stations_info.index.symmetric_difference(df.index)).sum()

169

In [25]:
stations_info.index.symmetric_difference(df.index).shape[0]

169

In [26]:
df.index.symmetric_difference(stations_info.index).shape[0] ==\
    stations_info.index.symmetric_difference(df.index).shape[0]

True